In [ ]:
!pip install mplfinance pytrends

In [ ]:
import requests
import mplfinance as mpf
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import tensorflow as tf
import time
from matplotlib import rc, style
from pytrends.request import TrendReq
from pytrends import dailydata
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV


def veri_al(kripto="ETH", birim="USD", aralik="day", aggregate=1):
    ex = birim
    aralik = aralik if aralik in ["day","hour","minute"] else "day" # day - hour - minute
    aggregate = aggregate # (saatlik ve dakikalık için 0-30)
    limitdata = 2000 if aralik=="day" else 2000//aggregate
    toTs= ""
    sorgu = f"&aggregate={aggregate}" if aralik != "day" else ""
    api_key = '' # cryptocompare api key
    api_url = f'https://min-api.cryptocompare.com/data/v2/histo{aralik}?limit={limitdata}{sorgu}&fsym={kripto}&tsym={ex}&api_key={api_key}'
    api_url_vol = f'https://min-api.cryptocompare.com/data/exchange/histo{aralik}?&limit={limitdata}{sorgu}&tsym={kripto}&api_key={api_key}'
    
    res = requests.get(api_url).json()
    
    df = pd.DataFrame(res['Data']['Data'])[['time', 'high', 'low', 'open', 'close']].set_index('time')
    df.index = pd.to_datetime(df.index, unit = 's')
    if aralik == "minute":
        df["volume"] = np.nan
    else:
        res2 = requests.get(api_url_vol).json()
        df2 = pd.DataFrame(res2["Data"]).set_index("time")
        df2.index = pd.to_datetime(df2.index, unit = 's')
        df = pd.concat([df,df2], axis=1)

    if aralik in ["hour","minute"]:
        for i in range(4 if aralik=="minute" else 7):
            timestamp = int(datetime.datetime.timestamp(df.index[0]))
            toTs= f"&toTs={timestamp}"
            api_url_toTs = f'https://min-api.cryptocompare.com/data/v2/histo{aralik}?limit={limitdata}{sorgu}&fsym={kripto}&tsym={ex}{toTs}&api_key={api_key}'
            api_url_vol_toTs = f'https://min-api.cryptocompare.com/data/exchange/histo{aralik}?&limit={limitdata}{sorgu}&tsym={kripto}{toTs}&api_key={api_key}'
            rests = requests.get(api_url_toTs).json()
            dfTs = pd.DataFrame(rests['Data']['Data'])[['time', 'high', 'low', 'open', 'close']].set_index('time')
            dfTs.index = pd.to_datetime(dfTs.index, unit = 's')
            if aralik == "minute":
                dfTs["volume"] = np.nan
            else:
                restsvol = requests.get(api_url_vol_toTs).json()
                dfTsVol = pd.DataFrame(restsvol["Data"]).set_index("time")
                dfTsVol.index = pd.to_datetime(dfTsVol.index, unit = 's')
                dfTs = pd.concat([dfTs,dfTsVol], axis=1)
            df = pd.concat([dfTs, df])
    return df

In [ ]:
kripto = "ETH"
data = veri_al(kripto=kripto, aralik="day")
data

In [ ]:
mpf.plot(data.tail(50), # 50 mumluk verinin mum grafiği
          type='candle',
          style='charles',
          ylabel='Fiyat $',
          volume=False if data["volume"].isna().any() else True,
          ylabel_lower='Hacim')

In [ ]:
sns.set()

In [ ]:
pytrend = TrendReq(tz=120)
pytrend.build_payload(kw_list=['bitcoin'], cat=0, timeframe='today 5-y')
ftrends = pytrend.interest_over_time()
ftrends = ftrends.resample('1d').pad().drop(['isPartial'], axis='columns')
ftrends.columns = ['frequency']

In [ ]:
data2 = pd.concat([data[["close"]][-180:], ftrends[-180:]], axis=1).dropna()
data2_scaled = data2.copy(deep=True)

In [ ]:
std_sc = StandardScaler()
data2_scaled['close'] = std_sc.fit_transform(data2_scaled['close'].values.reshape(-1,1))
data2_scaled['frequency'] = std_sc.fit_transform(data2_scaled['frequency'].values.reshape(-1,1))

In [ ]:
plt.plot(data2_scaled.index, data2_scaled['close'], color='blue', label=kripto)
plt.plot(data2_scaled.index, data2_scaled['frequency'], color='red', label="Google Aramaları (btc)")
plt.legend(loc='upper right')
plt.xlabel('Tarih')
plt.ylabel('Fiyat ve Arama Frekansı')
plt.show()

In [ ]:
ma50 = data.close.rolling(window=50).mean()
ma9 = data.close.rolling(window=9).mean()

In [ ]:
plt.plot(data.index[-360:], data.close[-360:], color='blue', label=kripto)
plt.plot(data.index[-360:], ma50[-360:], color='purple', label="MA50") # 50 mum hareketli ortalama
plt.plot(data.index[-360:], ma9[-360:], color='yellow', label="MA9") # 9 mum hareketli ortalama
plt.title("Fiyat ve Hareketli Ortalamalar")
plt.legend(loc="upper left")
plt.show()

In [ ]:
delta = data.close.diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
data["RSI"] = 100 - (100/(1+rs))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1], 'hspace': 0.1})
ax1.plot(data.index[-360:],data.close[-360:]) # 1 yıllık
ax2.plot(data.index[-360:],data.RSI[-360:], color="yellow")
ax1.set_ylabel(kripto+" Fiyat")
ax2.set_ylabel("RSI")
plt.show()

In [ ]:
close_data = data[["close"]].values
scaler = MinMaxScaler()
data_sc = scaler.fit_transform(close_data)

# Sadece kapanış fiyatları ile LSTM

In [ ]:
train_data, test_data = train_test_split(data_sc, test_size=0.2, shuffle=False)

In [ ]:
X_train, y_train = [], []
for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))

In [ ]:
X_test, y_test = [],[]
for i in range(60,len(test_data)):
    X_test.append(test_data[i-60:i,0])
    y_test.append(test_data[i,0])

X_test, y_test = np.array(X_test), np.array(y_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],1))
y_test = scaler.inverse_transform(y_test.reshape(-1,1))

In [ ]:
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")

In [ ]:
hist = model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
preds = model.predict(X_test)
preds = scaler.inverse_transform(preds)
plt.plot(y_test[-50:], color="b", label="Gerçek Değer")
plt.plot(preds[-50:], color="magenta", linestyle="--", label="Tahmin Edilen")
plt.legend(loc="upper left")
plt.show()

# En düşük, En yüksek, Açılış ve Kapanış Fiyatları Kullanılarak LSTM

In [ ]:
dataset = data.drop(["volume", "RSI"], axis=1)

In [ ]:
scaler2 = MinMaxScaler()

In [ ]:
dataset_sc = scaler2.fit_transform(dataset)

In [ ]:
train_data, test_data = train_test_split(dataset_sc, test_size=0.2, shuffle=False)

In [ ]:
X_train, y_train = [], []
for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i,:])
    y_train.append(train_data[i,0])

X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_test, y_test = [],[]
for i in range(60,len(test_data)):
    X_test.append(test_data[i-60:i,:])
    y_test.append(test_data[i,0])

X_test, y_test = np.array(X_test), np.array(y_test)
y_test = scaler.inverse_transform(y_test.reshape(-1,1))

In [ ]:
model = Sequential()
model.add(LSTM(units=100, return_sequences=False, input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1, activation="linear"))
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train,y_train, epochs=50, batch_size=32)

In [ ]:
model.save_weights("ETHUSD.h5")

In [ ]:
pred = model.predict(X_test)

In [ ]:
plt.plot(scaler.inverse_transform(pred.reshape(-1,1))[-180:], color="magenta", linestyle="--", label="tahmin")
plt.plot(y_test[-180:], color="b", label="gerçek değer")
plt.legend(loc="upper left")
plt.title("LSTM MODEL (Test Seti)")
plt.show()

In [ ]:
son60 = dataset_sc[-60:]

In [ ]:
pred2 = model.predict(son60.reshape(1,son60.shape[0],son60.shape[1]))

In [ ]:
scaler.inverse_transform(pred2.reshape(-1,1))

In [ ]:
data.close.values[-1]

# Lineer Regresyon ve ElasticNet

In [ ]:
X_train_2d = X_train.reshape((X_train.shape[0],X_train.shape[1]*X_train.shape[2]))
X_test_2d = X_test.reshape((X_test.shape[0],X_test.shape[1]*X_test.shape[2]))

In [ ]:
# Lineer Regresyon
reg = LinearRegression()

reg.fit(X_train_2d, y_train)

In [ ]:
pred = reg.predict(X_test_2d)

In [ ]:
plt.plot(scaler.inverse_transform(pred[-180:].reshape(-1,1)), color="magenta", linestyle="--", label="tahmin")
plt.plot(y_test[-180:], color="b", label="gerçek değer")
plt.legend(loc="upper left")
plt.title("Lineer Regresyon")
plt.show()

In [ ]:
# ElasticNet
elastic_cv = ElasticNetCV(cv = 10, random_state = 0).fit(X_train_2d, y_train)

elastic = ElasticNet(alpha=elastic_cv.alpha_)

elastic.fit(X_train_2d,y_train)

In [ ]:
pred = elastic.predict(X_test_2d)

In [ ]:
plt.plot(scaler.inverse_transform(pred[-180:].reshape(-1,1)), color="magenta", linestyle="--", label="tahmin")
plt.plot(y_test[-180:], color="b", label="gerçek değer")
plt.legend(loc="upper left")
plt.title("ElasticNet")
plt.show()